## Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/ColabNotebooks/Transformer


 LA-Transformer			'LA-Transformer Training.html'
 LA_Transformer.ipynb		'LA-Transformer Training.ipynb'
 LA_Transformer_Oneshot.ipynb	 LICENSE
'LA-Transformer Testing.html'	 Readme.md
'LA-Transformer Testing.ipynb'


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
!pip3 install timm faiss tqdm numpy
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!sudo apt-get install libomp-dev

     |████████████████████████████████| 431 kB 13.1 MB/s 
     |████████████████████████████████| 4.7 MB 31.8 MB/s 
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.3 MB/s eta 0:12:43tcmalloc: large alloc 1147494400 bytes == 0x55c1ea4ba000 @  0x7fc7c6910615 0x55c1b0eae3bc 0x55c1b0f8f18a 0x55c1b0eb11cd 0x55c1b0fa3b3d 0x55c1b0f25458 0x55c1b0f2002f 0x55c1b0eb2aba 0x55c1b0f252c0 0x55c1b0f2002f 0x55c1b0eb2aba 0x55c1b0f21cd4 0x55c1b0fa4986 0x55c1b0f21350 0x55c1b0fa4986 0x55c1b0f21350 0x55c1b0fa4986 0x55c1b0f21350 0x55c1b0eb2f19 0x55c1b0ef6a79 0x55c1b0eb1b32 0x55c1b0f251dd 0x55c1b0f2002f 0x55c1b0eb2aba 0x55c1b0f21cd4 0x55c1b0f2002f 0x55c1b0eb2aba 0x55c1b0f20eae 0x55c1b0eb29da 0x55c1b0f21108 0x55c1b0f2002f
     |██████████████████▌             | 1055.7 MB 1.3 MB/s eta 0:09:31tcmalloc: large alloc 1434370048 bytes == 0x55c22eb10000 @  0x7fc7c6910615 0x55c1b0eae3bc 0x55c1b0f8f18a 0x55c1b0eb11cd 0x55c1b0fa3b3d 0x55c1b0f2

In [6]:
import torch

print(f'torch.__version__             = {torch.__version__}')
print(f'torch.cuda.is_available()     = {torch.cuda.is_available()}')
print(f'torch.cuda.current_device()   = {torch.cuda.current_device()}')
print(f'torch.cuda.device(0)          = {torch.cuda.device(0)}')
print(f'torch.cuda.device_count()     = {torch.cuda.device_count()}')
print(f'torch.cuda.get_device_name(0) = {torch.cuda.get_device_name(0)}')

torch.__version__             = 1.10.2+cu113
torch.cuda.is_available()     = True
torch.cuda.current_device()   = 0
torch.cuda.device(0)          = <torch.cuda.device object at 0x7f17ea3237d0>
torch.cuda.device_count()     = 1
torch.cuda.get_device_name(0) = Tesla P100-PCIE-16GB


In [8]:
%cd /content/drive/MyDrive/ColabNotebooks/Transformer/LA-Transformer

/content/drive/.shortcut-targets-by-id/19RweVltTTlScqIDv6lHIQzlQezjmyFBN/ColabNotebooks/Transformer/LA-Transformer


# Testing

In [1]:
from __future__ import print_function

import os
import time
import glob
import random
import zipfile
from itertools import chain

import timm
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

import faiss

# from LATransformer.model import ClassBlock, LATransformer, LATransformerTest
# from LATransformer.utils import save_network, update_summary, get_id
# from LATransformer.metrics import rank1, rank5, rank10, calc_map

from osprey import LATransformerTest

def initilize_device(hardware):
    # os.environ['CUDA_VISIBLE_DEVICES']='1'
    if hardware == "gpu":
        device = "cuda"
    # if not device.type == "cpu":
        print(f'torch.__version__             = {torch.__version__}')
        print(f'torch.cuda.is_available()     = {torch.cuda.is_available()}')
        print(f'torch.cuda.current_device()   = {torch.cuda.current_device()}')
        print(f'torch.cuda.device(0)          = {torch.cuda.device(0)}')
        print(f'torch.cuda.device_count()     = {torch.cuda.device_count()}')
        print(f'torch.cuda.get_device_name(0) = {torch.cuda.get_device_name(0)}')
        
    elif hardware == "cpu":
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## Use if CPU
        print("Using cpu")

    else:
        print("Choose either gpu or cpu")
        return None

    return device

device = initilize_device("gpu")

torch.__version__             = 1.10.2+cu113
torch.cuda.is_available()     = True
torch.cuda.current_device()   = 0
torch.cuda.device(0)          = <torch.cuda.device object at 0x0000014EF0EA0E48>
torch.cuda.device_count()     = 1
torch.cuda.get_device_name(0) = NVIDIA GeForce GTX 1080


## Load Model

In [2]:
batch_size = 8
gamma = 0.7
seed = 42

# Load ViT
vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=14)
vit_base= vit_base.to(device)

# Create La-Transformer
osprey_model = LATransformerTest(vit_base, lmbd=8).to(device)

# Load LA-Transformer
# name = "la_with_lmbd_8"
name = "la_with_lmbd_8_12-03"
save_path = os.path.join('./model',name,'net_best.pth')

# # Load LA-Transformer
# name = "old_weights"
# save_path = os.path.join('./model',name,'small_ds_68_map_net_best.pth')

#Transformer\model\old_weights\small_ds_68_map_net_best.pth

osprey_model.load_state_dict(torch.load(save_path), strict=False)
# model.eval()

transform_query_list = [
    transforms.Resize((224,224), interpolation=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
transform_gallery_list = [
    transforms.Resize(size=(224,224),interpolation=3), #Image.BICUBIC
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
data_transforms = {
'query': transforms.Compose( transform_query_list ),
'gallery': transforms.Compose(transform_gallery_list),
}

E:\Anaconda\envs\py37\lib\site-packages\torchvision\transforms\transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


# Required functions

In [6]:

# device = initilize_device("cpu")
# We had to recreate the get_id() func since they assume the pictures are named in a specific manner. 
def get_id_padel(img_path):

  labels = []
  for path, v in img_path:
      filename = os.path.basename(path)

      label = filename.split('_')[0]
      labels.append(int(label))
  return labels
  
def extract_feature(model,dataloaders):
    
    features =  torch.FloatTensor()
    count = 0
    idx = 0
    for data in dataloaders:
        img, label = data
        img, label = img.to(device), label.to(device)

        output = model(img)

        n, c, h, w = img.size()
        
        count += n
        features = torch.cat((features, output.detach().cpu()), 0)
        idx += 1
    return features

def image_loader(data_dir_path):    
    image_datasets = {}
    # data_dir = "data/The_OspreyChallengerSet"
    data_dir = data_dir_path

    image_datasets['query'] = datasets.ImageFolder(os.path.join(data_dir, 'query'),
                                            data_transforms['query'])
    image_datasets['gallery'] = datasets.ImageFolder(os.path.join(data_dir, 'gallery'),
                                            data_transforms['gallery'])
    query_loader = DataLoader(dataset = image_datasets['query'], batch_size=batch_size, shuffle=False)
    gallery_loader = DataLoader(dataset = image_datasets['gallery'], batch_size=batch_size, shuffle=False)

    return query_loader, gallery_loader, image_datasets

def feature_extraction(model, query_loader, gallery_loader):
    # Extract Query Features
    query_feature = extract_feature(model, query_loader)

    # Extract Gallery Features
    gallery_feature = extract_feature(model, gallery_loader)

    return query_feature, gallery_feature

def get_labels(image_datasets):
    #Retrieve labels
    gallery_path = image_datasets['gallery'].imgs
    query_path = image_datasets['query'].imgs
    gallery_label = get_id_padel(gallery_path)
    query_label = get_id_padel(query_path)

    return gallery_label, query_label

def calc_gelt_feature(query_feature):
    concatenated_query_vectors = []
    for query in query_feature:  
        fnorm = torch.norm(query, p=2, dim=1, keepdim=True)*np.sqrt(14) 
        query_norm = query.div(fnorm.expand_as(query))        
        concatenated_query_vectors.append(query_norm.view((-1))) # 14*768 -> 10752
    return concatenated_query_vectors

def calc_gelt_gallery(gallery_feature):
    concatenated_gallery_vectors = []
    for gallery in gallery_feature:  
        fnorm = torch.norm(gallery, p=2, dim=1, keepdim=True) *np.sqrt(14)  
        gallery_norm = gallery.div(fnorm.expand_as(gallery))      
        concatenated_gallery_vectors.append(gallery_norm.view((-1))) # 14*768 -> 10752  
    return concatenated_gallery_vectors

def calc_faiss(concatenated_gallery_vectors, gallery_label):
    index = faiss.IndexIDMap(faiss.IndexFlatIP(10752))
    index.add_with_ids(np.array([t.numpy() for t in concatenated_gallery_vectors]), np.array(gallery_label).astype('int64'))  # original 
    return index

def search(query: str, k=1):
    encoded_query = query.unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, k)
    return top_k

In [7]:
def osprey_detect(data_dir_path, osprey_model):

    query_loader, gallery_loader, image_datasets  = image_loader(data_dir_path=data_dir_path)

    query_feature, gallery_feature = feature_extraction(model=osprey_model, query_loader=query_loader, gallery_loader=gallery_loader)

    gallery_label, query_label = get_labels(image_datasets)

    concatenated_query_vectors = calc_gelt_feature(query_feature)
    concatenated_gallery_vectors = calc_gelt_gallery(gallery_feature)

    index = calc_faiss(concatenated_gallery_vectors, gallery_label)

    return concatenated_query_vectors, index

In [10]:
concatenated_query_vectors, index = osprey_detect("data/bim_bam", osprey_model)

In [11]:
#For each vector in the query vector list

for query in concatenated_query_vectors:
    output = search(query)
    print(f"Predicted class: {output[1][0][0]} with {output[0][0][0] * 100} % confidence")

Predicted class: 3 with 68.69755983352661 % confidence
Predicted class: 4 with 62.87815570831299 % confidence


In [7]:
##Making new class boy
def predictClass(queryVector):
    output = search(queryVector)
    print(f"Predicted class: {output[1][0][0]} with {output[0][0][0] * 100} % confidence")

    return output[1][0][0]

odoijadsoijas

In [5]:
#query_loader, gallery_loader, image_datasets  = image_loader(data_dir_path="data/The_OspreyChallengerSet")

#load images from folder
query_loader, gallery_loader, image_datasets  = image_loader(data_dir_path="data/bim_bam")

#extract features
query_feature, gallery_feature = feature_extraction(model=osprey_model, query_loader=query_loader)

#get labels from pictures
gallery_label, query_label = get_labels(image_datasets)


concatenated_query_vectors = calc_gelt_feature(query_feature)
concatenated_gallery_vectors = calc_gelt_gallery(gallery_feature)


index = calc_faiss(concatenated_gallery_vectors, gallery_label)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

<faiss.swigfaiss.IndexIDMap; proxy of <Swig Object of type 'faiss::IndexIDMap *' at 0x7f17c243ebd0> >


In [38]:
rank1_score = 0
rank5_score = 0
rank10_score = 0
ap = 0
count = 0
for query, label in zip(concatenated_query_vectors, query_label):
    count += 1
    label = label
    output = search(query, k=10)
#     print(output)
    rank1_score += rank1(label, output) 
    rank5_score += rank5(label, output) 
    rank10_score += rank10(label, output) 
    print("Correct: {}, Total: {}, Incorrect: {}".format(rank1_score, count, count-rank1_score), end="\r")
    ap += calc_map(label, output)

print("Rank1: {}, Rank5: {}, Rank10: {}, mAP: {}".format(rank1_score/len(query_feature), 
                                                         rank5_score/len(query_feature), 
                                                         rank10_score/len(query_feature), ap/len(query_feature))) 

Rank1: 1.0, Rank5: 1.0, Rank10: 1.0, mAP: 0.6766666666666666
